## Programa que retorna o valor líquido, dada uma base inicial

In [1]:
# import base do pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, format_number, round as spark_round

sc = SparkContext()
sqlContext = SQLContext(sc)

In [2]:
#constante simulando dados de entrada
TRANSACTIONS = [
    {'transacao_id': 1, 'total_bruto': 3000,  'desconto_percentual': 6.99},
    {'transacao_id': 2, 'total_bruto': 57989, 'desconto_percentual': 1.45},
    {'transacao_id': 4, 'total_bruto': 1,     'desconto_percentual': None},
    {'transacao_id': 5, 'total_bruto': 34,    'desconto_percentual': 0.0 }
]

### Transformando a lista de dados em DataFrame e organizando as colunas na ordem convencional

A ordem das colunas fica diferente quando transformamos uma lista de `dict` em DataFrame pois `dict` é uma estrutura não-ordenada. Para termos consistência na ordem, basta selecionar as colunas que precisamos com a função _select_ do spark sql. 

In [3]:
input_transactions = sqlContext.createDataFrame(TRANSACTIONS)
ordered_transactions = input_transactions.select('transacao_id', 'total_bruto', 'desconto_percentual')

In [4]:
ordered_transactions.show()

+------------+-----------+-------------------+
|transacao_id|total_bruto|desconto_percentual|
+------------+-----------+-------------------+
|           1|       3000|               6.99|
|           2|      57989|               1.45|
|           4|          1|               null|
|           5|         34|                0.0|
+------------+-----------+-------------------+



### Tratando valores nulos

Em um caso com muitas colunas de tipos diferentes com valores nulos e não houvesse necessidade de transformá-los, eu escolheria tratar os nulos na própria expressão , no passo seguinte.

In [5]:
transactions_notnull = ordered_transactions.fillna(0.0, subset=["desconto_percentual", "total_bruto"])

In [6]:
transactions_notnull.show()

+------------+-----------+-------------------+
|transacao_id|total_bruto|desconto_percentual|
+------------+-----------+-------------------+
|           1|       3000|               6.99|
|           2|      57989|               1.45|
|           4|          1|                0.0|
|           5|         34|                0.0|
+------------+-----------+-------------------+



### Cálculo de valor líquido

Para o cálculo de valor total líquido, utilizei as funções withColumn para criar uma coluna nova, col para acessar as colunas já existentres no DataFrame e a _round_ do spark para deixar os dados com duas casas decimais. 

In [7]:
transactions_liquido = transactions_notnull.withColumn(
    "total_liquido", 
    spark_round((col("total_bruto") - (col("total_bruto") * (col("desconto_percentual") / 100))), 2)
)

In [8]:
transactions_liquido.show()

+------------+-----------+-------------------+-------------+
|transacao_id|total_bruto|desconto_percentual|total_liquido|
+------------+-----------+-------------------+-------------+
|           1|       3000|               6.99|       2790.3|
|           2|      57989|               1.45|     57148.16|
|           4|          1|                0.0|          1.0|
|           5|         34|                0.0|         34.0|
+------------+-----------+-------------------+-------------+



### Agregação e função soma

Para retornar o valor total líquido, apenas precisamos utilizar a função de soma para a coluna _total_liquido_, não tendo necessidade de um _group by_

In [9]:
total_liquido = transactions_liquido.agg({'total_liquido':'sum'}).select(format_number("sum(total_liquido)", 2).alias("total_liquido"))
total_liquido.show()

+-------------+
|total_liquido|
+-------------+
|    59,973.46|
+-------------+

